## Libraries

In [1]:
import pandas as pd
import requests, zipfile, io
from pathlib import Path
import shutil
import xlwt
import xlrd
import xlsxwriter
import datetime

## Data Extraction

Download all the data from http://www.mercadocentral.gob.ar/sites/default/files/precios_mayoristas/PM-Hortalizas-2019.zip
and unzip files to data/raw folder

In [16]:
# Download data from Mercado Central
zip_file_url = 'http://www.mercadocentral.gob.ar/sites/default/files/precios_mayoristas/PM-Hortalizas-2019.zip'
path_to_extract_to = 'data/zip'
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path_to_extract_to)

In [17]:
# Unzip all files
paths = Path(Path.cwd().joinpath('data','zip')).glob('*.zip')
for path in paths:
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall('data/raw/vegetables')
        
# Delete unused files
shutil.rmtree('data/zip')

## From XLS to Pandas

In [5]:
# Function that takes a month in spanish and transform to month number
def mesANumero(string):
    m = {
        'ene': "01",
        'feb': "02",
        'marzo': "03",
        'abril': "04",
        'mayo': "05",
        'junio': "06",
        'julio': "07",
        'agosto': "08",
        'sep': "09",
        'oct': "10",
        'nov': "11",
        'dic': "12"
        }

    fecha = string.split("-")
    dia =  fecha[0]
    mes =  fecha[1]
    anio = fecha[2]
    out = str(m[mes.lower()])
    return (dia + "-" + out + "-" + anio)

In [21]:
def generate_dataframe(datafile):
    
    global df
    
    # Read XLS with awful encoding
    workbook = xlrd.open_workbook(datafile, encoding_override='cp1252')
    sheet = workbook.sheet_by_index(0)

    # rows as lists
    esp_row = [sheet.cell_value(row, 0) for row in range(1,sheet.nrows)]
    mapk_row = [sheet.cell_value(row, 11) for row in range(1,sheet.nrows)]
    mipk_row = [sheet.cell_value(row, 13) for row in range(1,sheet.nrows)]

    # Create Dataframe
    df = pd.DataFrame(list(zip(esp_row,mapk_row,mipk_row)), columns = ['item','max_price_per_kg','min_price_per_kg'])

    # Data transformation
    df['average_kg_price'] = df.mean(axis=1)
    df = df[df.average_kg_price > 0] # Delete items with price = 0
    df = df.groupby('item', as_index = False).mean()
    
    # Read file path and get filename
    #path = Path(Path.cwd().joinpath('data','raw',file.stem))
    filename = file.stem

    # Identify date
    day = filename[14:16]
    month = filename[17:filename.find('-2019',13)]
    year = filename[-4:]

    # Create dates and add to dataframe
    base = mesANumero(day +'-'+ month + '-' + year)
    date_list = [base for x in range(len(df))]
    df['date'] = date_list

    # Reareange columns
    df = df[['date', 'item', 'average_kg_price']]
    
    return (df.size)

In [22]:
folder = Path(Path.cwd().joinpath('data','raw','vegetables')).glob('**/*')
dataset = pd.DataFrame()

for file in folder:
    try:
        generate_dataframe(file)
        dataset = dataset.append(df)
    except:
        print(f'File that crashed = {file.stem}')

In [23]:
filename = Path(Path.cwd().joinpath('data','dataset_vegetables.csv'))

dataset.to_csv(filename,index = False)

In [15]:
folder

<generator object Path.glob at 0x115f8d7d8>

In [ ]:
pd.read_